# Preparing the IMDb dataset.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import requests
import time

api_key1 = "b58bb13f"
api_key2 = "9bf3218a"

RANDOM_STATE = 1212

##
asd = pd.DataFrame(columns=["Title", "Year", "Genre", "Plot"])
##

In [7]:
# This block can be run to create the necessary csv files for the first time

""" not_found = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])
imdb_data = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])

not_found.to_csv("not_found_NEW.csv", index=False)
imdb_data.to_csv("imdb_data_NEW.csv", index=False) """

' not_found = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])\nimdb_data = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])\n\nnot_found.to_csv("not_found_NEW.csv", index=False)\nimdb_data.to_csv("imdb_data_NEW.csv", index=False) '

In [8]:
not_found = pd.read_csv("not_found_NEW.csv")
imdb_data = pd.read_csv("imdb_data_NEW.csv")
cmu_data = pd.read_csv("cmu_backup.csv")

for index, row in cmu_data.iterrows():
    if len(imdb_data[imdb_data["genre"] == row["genre"]]) >= 1500:
        continue

    if row["movie_name"] in not_found["movie_name"].values and row["year"] in not_found["year"].values and row["genre"] in not_found["genre"].values:
        continue
    if row["movie_name"] in imdb_data["movie_name"].values and row["year"] in imdb_data["year"].values and row["genre"] in imdb_data["genre"].values:
        continue

    url = f"http://www.omdbapi.com/?t={row['movie_name']}&y={row['year']}&plot=full&apikey={api_key2}"
    response = requests.get(url)
    data = response.json()

    time.sleep(0.5)

    if data["Response"] == "False":
        if data["Error"] == "Request limit reached!":
            print(data["Error"])
            break
        if data["Error"] == "Movie not found!":
            url = f"http://www.omdbapi.com/?t={row['movie_name']}&y={int(row['year']) - 1}&plot=full&apikey={api_key1}"            
            response = requests.get(url)
            data = response.json()
            time.sleep(0.5)
            if data["Response"] == "True":
                if data["Plot"].count(".") >= 2:
                    imdb_data.loc[len(imdb_data)] = [data['Title'], int(data['Year'][:4]), row['genre'], data['Plot']]
                else:
                    not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], data['Plot']]
            else:
                not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], row['plot']]
        else: 
            not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], row['plot']]
    else:
        if data["Plot"].count(".") >= 2:
            imdb_data.loc[len(imdb_data)] = [data['Title'], int(data['Year'][:4]), row['genre'], data['Plot']]
        else:
            not_found.loc[len(not_found)] = [row["movie_name"], int(row["year"]), row['genre'], data['Plot']]


not_found.to_csv("not_found_NEW.csv", index=False)
imdb_data.to_csv("imdb_data_NEW.csv", index=False)

Request limit reached!


# Removing movies that were not found from the CMU dataset and filling it back up

## WIP, don't run these

In [9]:
""" # Merge the dataframes on movie_name and year
merged = pd.merge(cmu_data, not_found, on=['movie_name', 'year', 'genre'], how='left', indicator=True)

# Filter out rows where movie_name and year match
filtered_data = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

# get the genres that were removed
genres = not_found["genre"].unique()

# get the genres that were removed and the number of items in each genre
genre_counts = not_found["genre"].value_counts()

print(genre_counts)




# merged.shape, filtered_data.shape """

' # Merge the dataframes on movie_name and year\nmerged = pd.merge(cmu_data, not_found, on=[\'movie_name\', \'year\', \'genre\'], how=\'left\', indicator=True)\n\n# Filter out rows where movie_name and year match\nfiltered_data = merged[merged[\'_merge\'] == \'left_only\'].drop(columns=[\'_merge\'])\n\n# get the genres that were removed\ngenres = not_found["genre"].unique()\n\n# get the genres that were removed and the number of items in each genre\ngenre_counts = not_found["genre"].value_counts()\n\nprint(genre_counts)\n\n\n\n\n# merged.shape, filtered_data.shape '

In [10]:
""" comedy_movies = filtered_data[filtered_data["genre"].str.contains("Comedy")]
horror_movies = filtered_data[filtered_data["genre"].str.contains("Horror")]
thriller_movies = filtered_data[filtered_data["genre"].str.contains("Thriller")]
drama_movies = filtered_data[filtered_data["genre"].str.contains("Drama")]

# add 1500 items to each genre
# if the genre has less than 1500 items, add items from cmu_backups
# if adding a comedy movie, make sure it has no horror, thriller or drama in the genre

filtered_comedy = cmu_backups[cmu_backups['genre'].apply(lambda x: 'Comedy' in x and not any(genre in x for genre in ['Horror', 'Thriller', 'Drama']))]
comedy_movies = pd.concat([comedy_movies, filtered_comedy])
comedy_movies = comedy_movies.head(1500)

filtered_horror = cmu_backups[cmu_backups['genre'].apply(lambda x: 'Horror' in x and not any(genre in x for genre in ['Comedy', 'Thriller', 'Drama']))]
horror_movies = pd.concat([horror_movies, filtered_horror])
horror_movies = horror_movies.head(1500)

filtered_thriller = cmu_backups[cmu_backups['genre'].apply(lambda x: 'Thriller' in x and not any(genre in x for genre in ['Comedy', 'Horror', 'Drama']))]
thriller_movies = pd.concat([thriller_movies, filtered_thriller])
thriller_movies = thriller_movies.head(1500)

filtered_drama = cmu_backups[cmu_backups['genre'].apply(lambda x: 'Drama' in x and not any(genre in x for genre in ['Comedy', 'Horror', 'Thriller']))]
drama_movies = pd.concat([drama_movies, filtered_drama])
drama_movies = drama_movies.head(1500)

cmu_dataset = pd.concat([comedy_movies, horror_movies, thriller_movies, drama_movies])
cmu_dataset.reset_index(drop=True, inplace=True)

cmu_backups = cmu_backups[~cmu_backups.isin(cmu_dataset)].dropna()

cmu_dataset.to_csv("cmu_dataset_v2.csv", index=False)
cmu_backups.to_csv("cmu_movies_backup.csv", index=False)

cmu_dataset.shape, cmu_backups.shape, comedy_movies.shape, horror_movies.shape, thriller_movies.shape, drama_movies.shape """

' comedy_movies = filtered_data[filtered_data["genre"].str.contains("Comedy")]\nhorror_movies = filtered_data[filtered_data["genre"].str.contains("Horror")]\nthriller_movies = filtered_data[filtered_data["genre"].str.contains("Thriller")]\ndrama_movies = filtered_data[filtered_data["genre"].str.contains("Drama")]\n\n# add 1500 items to each genre\n# if the genre has less than 1500 items, add items from cmu_backups\n# if adding a comedy movie, make sure it has no horror, thriller or drama in the genre\n\nfiltered_comedy = cmu_backups[cmu_backups[\'genre\'].apply(lambda x: \'Comedy\' in x and not any(genre in x for genre in [\'Horror\', \'Thriller\', \'Drama\']))]\ncomedy_movies = pd.concat([comedy_movies, filtered_comedy])\ncomedy_movies = comedy_movies.head(1500)\n\nfiltered_horror = cmu_backups[cmu_backups[\'genre\'].apply(lambda x: \'Horror\' in x and not any(genre in x for genre in [\'Comedy\', \'Thriller\', \'Drama\']))]\nhorror_movies = pd.concat([horror_movies, filtered_horror])\